In [81]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from glob import glob
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.team_predicter import TeamPredicter, SolverMode

import config

In [82]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./data/results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 0
TOTAL_BUDGET = 1000

In [83]:
current_team_names = {"Jordan Pickford",
                      "Matz Sels",
                      "Dan Burn",
                      "Michael Keane",
                      "Wout Faes",
                      "Jack Stephens",
                      "Ola Aina",
                      "Bryan Mbeumo",
                      "Cole Palmer",
                      "James Maddison",
                      "Alex Iwobi",
                      "Mohamed Salah",
                      "Yoane Wissa",
                      "Alexander Isak",
                      "Chris Wood"
                      }

In [84]:
def getDataFilesSorted():
    allDataFiles = sorted(glob(r"./data/player_stats/data_*.csv"))
    filesSorted = []
    regex = r"^(\.\/data\/player_stats\\data_)(\d+)(\.csv)"
    pattern = re.compile(regex)
    for file in allDataFiles:
        fileNameSplit = re.split(pattern,file)
        if (len(fileNameSplit) == 5):
            fileNum = fileNameSplit[2]
            dictObj = {
                "name": file,
                "number": int(fileNum)
            }
            filesSorted.append(dictObj)
    filesSorted.sort(key=lambda x: x["number"])
    return [file["name"] for file in filesSorted]

In [85]:
filesSorted = getDataFilesSorted()
dataFileName = filesSorted[-1]

In [86]:
with open(dataFileName,"r",encoding="utf-8") as f:
    all_player_data = pd.read_csv(f)

In [87]:
def calcPScores(pSeries: pd.Series) -> pd.Series:
    stdDev = np.std(pSeries)
    avg = pSeries.mean()
    return (pSeries - avg) / stdDev

ictIndexPScores = calcPScores(all_player_data["ict_index"])
avgPointsPScores = calcPScores(all_player_data["points_per_game"])
totalPointsPScores = calcPScores(all_player_data["total_points"])

combinedPScores = ictIndexPScores + avgPointsPScores + totalPointsPScores
all_player_data["combined"] = combinedPScores

In [88]:
print(all_player_data)

      id                          name  cost  ict_index  total_points  \
0      0         Fábio Ferreira Vieira    54        0.0             0   
1      1     Gabriel Fernando de Jesus    68       52.6            42   
2      2  Gabriel dos Santos Magalhães    64       82.5            84   
3      3                   Kai Havertz    78      120.2            75   
4      4                     Karl Hein    40        0.0             0   
..   ...                           ...   ...        ...           ...   
719  719   Carlos Roberto Forbs Borges    55        8.3             8   
720  720                    Alfie Pond    39        0.0             1   
721  721                    Tom Edozie    45        0.0             0   
722  722                   Wes Okoduwa    40        0.0             0   
723  723              Emmanuel Agbadou    40        2.3             0   

     points_per_game  form status  starts_per_90 position team  combined  
0                0.0   0.0      u           0.00

In [89]:
current_team = Team.fromNameSet(current_team_names, all_player_data)

In [90]:
current_team_cost = current_team.getTotalCost()
current_team_cost

1005

In [91]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)
data = all_data["data"][SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame(data=data)

In [92]:
selectedTeamNames = selected_team_df["name"].values
selected_team = Team.fromNameSet(selectedTeamNames, all_player_data)

In [93]:
display(selected_team)

In [94]:
new_players = selected_team - current_team
print(new_players)


Total Score: 0

Goalkeepers:
- André Onana (Captain) 	Score: 0	Cost: 51	Fixture Difficulty: 0.0
Defenders:
- Harry Clarke (Vice Captain) 	Score: 0	Cost: 40	Fixture Difficulty: 0.0
- Marc Guéhi	Score: 0	Cost: 45	Fixture Difficulty: 0.0
Attackers:
- Erling Haaland	Score: 0	Cost: 148	Fixture Difficulty: 0.0
- Danny Welbeck	Score: 0	Cost: 55	Fixture Difficulty: 0.0
- Michail Antonio	Score: 0	Cost: 53	Fixture Difficulty: 0.0
Midfielders:
- Dwight McNeil	Score: 0	Cost: 51	Fixture Difficulty: 0.0
- Mikkel Damsgaard	Score: 0	Cost: 50	Fixture Difficulty: 0.0


In [95]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -9


In [96]:
HEURISTIC = "combined"
MODE = SolverMode.CHEAPEST_FIRST

In [97]:
teamPredicter = TeamPredicter(HEURISTIC, MODE, True)

[DEBUG]: Reading from data files...
[DEBUG]: Done reading data files! Calculating linear regression...
Done calculating linear regression!


In [98]:
all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(12.789356888941393, 12.789356888941393)

In [99]:
START_SAMPLE_GAMEWEEK = 17
END_SAMPLE_GAMEWEEK = 20

In [100]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK)

new_players.recalculateFixtureDifficulty(matrix)
new_players.calculateScore(HEURISTIC)

selected_team.recalculateFixtureDifficulty(matrix)
selected_team.calculateScore(HEURISTIC)

current_team.recalculateFixtureDifficulty(matrix)
current_team.calculateScore(HEURISTIC)

In [101]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [102]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: -298.10909151531035


In [103]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [104]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [105]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [106]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [107]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team):
    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    return newTeam

In [108]:
newTeam = getNewTeam(current_team, selected_team, new_players)
newTeam = newTeam.toBenchTeam()
display(newTeam)

Best transfer:
Transfer from James Maddison -> Mikkel Damsgaard:
Old player: James Maddison	Score: 8.33	Cost: 75	Fixture Difficulty: 5.499423462244801
New player: Mikkel Damsgaard	Score: 17.39	Cost: 50	Fixture Difficulty: 1.9184035333412073
- Cost change: -25
- Score change: 9.06
